In [76]:
import datetime
import numpy as np
#import scipy as sp
#import uuid
import hashlib
import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import os
import re
import math
from datetime import date
import ftplib
from ftplib import FTP
import gzip as gz
import collections
from collections import deque,OrderedDict
import sys
import cProfile
import multiprocessing
import inspect
#https://github.com/chaithemleib/intervaltree - don't use one that cannot support objects attached to intervals
import yaml
import datetime
import ast
import codecs
import uuid
import json
# add liftover
from pyliftover import LiftOver
import hgvs.parser
import hgvs.dataproviders.uta
import hgvs.assemblymapper
import hgvs.projector

In [9]:
hgvsparser = hgvs.parser.Parser()

In [16]:
hdp = hgvs.dataproviders.uta.connect()

In [17]:
am = hgvs.assemblymapper.AssemblyMapper(hdp,assembly_name='GRCh37', alt_aln_method='splign',replace_reference=True)

In [18]:
hgvs_g = 'NC_000007.13:g.36561662C>T'

In [19]:
hgvs_c = 'NM_001637.3:c.1582G>A'

In [20]:
hp = hgvs.parser.Parser()

In [21]:
var_g = hp.parse_hgvs_variant(hgvs_g)

In [22]:
var_g

SequenceVariant(ac=NC_000007.13, type=g, posedit=36561662C>T, gene=None)

In [24]:
var_g.posedit.pos.start

SimplePosition(base=36561662, uncertain=False)

In [25]:
str(var_g)

'NC_000007.13:g.36561662C>T'

In [26]:
var_g.posedit.pos.start

SimplePosition(base=36561662, uncertain=False)

In [27]:
str(var_g)

'NC_000007.13:g.36561662C>T'

In [28]:
hdp = hgvs.dataproviders.uta.connect()

In [29]:
am = hgvs.assemblymapper.AssemblyMapper(hdp,assembly_name='GRCh37', alt_aln_method='splign',replace_reference=True)

In [30]:
transcripts = am.relevant_transcripts(var_g)

In [31]:
sorted(transcripts)

['NM_001177506.1', 'NM_001177507.1', 'NM_001637.3']

In [32]:
var_c = am.g_to_c(var_g, 'NM_001637.3')

In [33]:
var_c

SequenceVariant(ac=NM_001637.3, type=c, posedit=1582G>A, gene=None)

In [51]:
var_c2 = am.g_to_c(var_g, 'NM_001177507.1')
var_c2

SequenceVariant(ac=NM_001177507.1, type=c, posedit=1486G>A, gene=None)

In [54]:
var_c3 = am.g_to_c(var_g,'NM_001177506.1')
var_c3

SequenceVariant(ac=NM_001177506.1, type=c, posedit=1582G>A, gene=None)

In [34]:
str(var_c)

'NM_001637.3:c.1582G>A'

In [55]:
str(var_c2)

'NM_001177507.1:c.1486G>A'

In [59]:
str(var_c3)

'NM_001177506.1:c.1582G>A'

In [60]:
dir(var_c3)

['__attrs_attrs__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'ac',
 'fill_ref',
 'format',
 'gene',
 'posedit',
 'type',
 'validate']

In [63]:
var_c3.validate

<bound method SequenceVariant.validate of SequenceVariant(ac=NM_001177506.1, type=c, posedit=1582G>A, gene=None)>

In [73]:
vr = hgvs.validator.Validator(hdp=hdp)
try:
    # shift the NM_001177506.1 position +2 and see if this validates
    vr.validate( hp.parse_hgvs_variant('NM_001177506.1:c.1584G>A') )
    vr.validate( hp.parse_hgvs_variant('NM_001637.3:c.1582G>A') )
except hgvs.exceptions.HGVSError as e:
    print(e)

NM_001177506.1:c.1584G>A: Variant reference (G) does not agree with reference sequence (A)


In [36]:
var_p = am.c_to_p(var_c)

In [37]:
var_p

SequenceVariant(ac=NP_001628.1, type=p, posedit=(Gly528Arg), gene=None)

In [38]:
 str(var_p)

'NP_001628.1:p.(Gly528Arg)'

In [39]:
var_p.posedit.uncertain = False

In [40]:
str(var_p)

'NP_001628.1:p.Gly528Arg'

In [41]:
 var_p.format(conf={"p_3_letter": False})

'NP_001628.1:p.G528R'

In [42]:
hn = hgvs.normalizer.Normalizer(hdp)

In [43]:
hn.normalize(hp.parse_hgvs_variant('NM_001166478.1:c.35_36insT'))

SequenceVariant(ac=NM_001166478.1, type=c, posedit=35dup, gene=None)

In [44]:
c1 = hp.parse_hgvs_variant('NM_001166478.1:c.31del')

In [45]:
c1

SequenceVariant(ac=NM_001166478.1, type=c, posedit=31del, gene=None)

In [46]:
c1n = hn.normalize(c1)

In [47]:
 g = am.c_to_g(c1)

In [48]:
g

SequenceVariant(ac=NC_000006.11, type=g, posedit=49917127del, gene=None)

In [49]:
c2 = am.g_to_c(g, c1.ac)

In [50]:
c2

SequenceVariant(ac=NM_001166478.1, type=c, posedit=35del, gene=None)

In [74]:
var_c1 = hgvsparser.parse_hgvs_variant('NM_001261456.1:c.1762A>G')

In [77]:
pj = hgvs.projector.Projector(hdp=hdp,
                              alt_ac='NC_000001.10',
                              src_ac=var_c1.ac,
                              dst_ac='NM_001261457.1')

In [78]:
pj.project_variant_forward(var_c1)

SequenceVariant(ac=NM_001261457.1, type=c, posedit=1534A>G, gene=None)

In [79]:
hdp = hgvs.dataproviders.uta.connect()
variantmapper = hgvs.variantmapper.VariantMapper(hdp)
hgvsparser = hgvs.parser.Parser()

In [80]:
var_c1 = hgvsparser.parse_hgvs_variant('NM_001261456.1:c.1762A>G')
var_p1 = variantmapper.c_to_p(var_c1, None)
var_c1, var_p1

(SequenceVariant(ac=NM_001261456.1, type=c, posedit=1762A>G, gene=None),
 SequenceVariant(ac=NP_001248385.1, type=p, posedit=(Met588Val), gene=None))

In [81]:
var_g = variantmapper.c_to_g(var_c1,'NC_000001.10')
var_g

SequenceVariant(ac=NC_000001.10, type=g, posedit=160793560A>G, gene=None)

In [82]:
txs = hdp.get_tx_for_gene('LY9')
txs

[['LY9', 114, 2040, 'NM_001261456.1', 'AC_000133.1', 'splign'],
 ['LY9', 114, 2040, 'NM_001261456.1', 'NC_000001.10', 'blat'],
 ['LY9', 114, 2040, 'NM_001261456.1', 'NC_000001.10', 'splign'],
 ['LY9', 114, 2040, 'NM_001261456.1', 'NC_000001.11', 'splign'],
 ['LY9', 114, 2040, 'NM_001261456.1', 'NC_018912.2', 'splign'],
 ['LY9', 114, 2040, 'ENST00000368037', 'NC_000001.10', 'genebuild'],
 ['LY9', 114, 696, 'NM_001033667.2', 'AC_000133.1', 'splign'],
 ['LY9', 114, 696, 'NM_001033667.2', 'NC_000001.10', 'blat'],
 ['LY9', 114, 696, 'NM_001033667.2', 'NC_000001.10', 'splign'],
 ['LY9', 114, 696, 'NM_001033667.2', 'NC_000001.11', 'splign'],
 ['LY9', 114, 696, 'NM_001033667.2', 'NC_018912.2', 'splign'],
 ['LY9', 114, 2082, 'NM_002348.3', 'AC_000133.1', 'splign'],
 ['LY9', 114, 2082, 'NM_002348.3', 'NC_000001.10', 'blat'],
 ['LY9', 114, 2082, 'NM_002348.3', 'NC_000001.10', 'splign'],
 ['LY9', 114, 2082, 'NM_002348.3', 'NC_000001.11', 'splign'],
 ['LY9', 114, 2082, 'NM_002348.3', 'NC_018912.2',

In [83]:
txs = hdp.get_tx_for_gene('ABCA4')
txs

[['ABCA4', 87, 6909, 'ENST00000370225', 'NC_000001.10', 'genebuild'],
 ['ABCA4', None, None, 'ENST00000465352', 'NC_000001.10', 'genebuild'],
 ['ABCA4', None, None, 'ENST00000484388', 'NC_000001.10', 'genebuild'],
 ['ABCA4', None, None, 'ENST00000470771', 'NC_000001.10', 'genebuild'],
 ['ABCA4', None, None, 'ENST00000460514', 'NC_000001.10', 'genebuild'],
 ['ABCA4', None, None, 'ENST00000472033', 'NC_000001.10', 'genebuild'],
 ['ABCA4', 93, 1725, 'ENST00000536513', 'NC_000001.10', 'genebuild'],
 ['ABCA4', 225, 1404, 'ENST00000535881', 'NC_000001.10', 'genebuild'],
 ['ABCA4', 80, 3020, 'ENST00000535735', 'NC_000001.10', 'genebuild'],
 ['ABCA4', 104, 6926, 'NM_000350.2', 'AC_000133.1', 'splign'],
 ['ABCA4', 104, 6926, 'NM_000350.2', 'NC_000001.10', 'blat'],
 ['ABCA4', 104, 6926, 'NM_000350.2', 'NC_000001.10', 'splign'],
 ['ABCA4', 104, 6926, 'NM_000350.2', 'NC_000001.11', 'splign'],
 ['ABCA4', 104, 6926, 'NM_000350.2', 'NC_018912.2', 'splign'],
 ['ABCA4', 104, 6926, 'NM_000350.2', 'NG_00

In [84]:
var_c2 = variantmapper.g_to_c(var_g,'NM_001261457.1',alt_aln_method='splign')
var_p2 = variantmapper.c_to_p(var_c2, None)
var_c2, var_p2

(SequenceVariant(ac=NM_001261457.1, type=c, posedit=1534A>G, gene=None),
 SequenceVariant(ac=NP_001248386.1, type=p, posedit=(Met512Val), gene=None))

In [85]:
v = hp.parse_hgvs_variant("NM_001166478.1:c.30_31insT")
str(hn.normalize(v))

'NM_001166478.1:c.35dup'